# Analysis of `summaries.txt` information

- number of raw files (no here)
- number of raw files with MQ-Output
- MS1 per file
- MS2 per file

In [1]:
import ipywidgets as widgets
import yaml
import numpy as np
from vaep.io.data_objects import MqAllSummaries
import vaep

from src.config import FN_ALL_SUMMARIES

mq_all_summaries = MqAllSummaries(FN_ALL_SUMMARIES)
mq_all_summaries.df.describe().T

FOLDER_MQ_TXT_DATA = data\mq_out
MqAllSummaries: Load summaries of 6 folders.


,count,mean,std,min,25%,50%,75%,max
Enzyme first search,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Enzyme mode first search,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Multi modifications,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Variable modifications first search,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Multiplicity,6.000,1.000,0.000,1.000,1.000,1.000,1.000,1.000
Max. missed cleavages,6.000,2.000,0.000,2.000,2.000,2.000,2.000,2.000
Labels0,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
Time-dependent recalibration,0,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
MS,6.000,"13,430.000","2,416.218","11,134.000","11,906.750","12,344.500","15,377.250","16,608.000"
MS/MS,6.000,"133,727.333","17,423.545","105,883.000","127,771.000","132,453.500","147,882.000","152,339.000"


In [2]:
FN_ALL_SUMMARIES: str = FN_ALL_SUMMARIES

Find unique columns, see [post](https://stackoverflow.com/a/54405767/9684872)

In [ ]:
from vaep.pandas import unique_cols
unique_cols(mq_all_summaries.df.Multiplicity), unique_cols(
    mq_all_summaries.df["Variable modifications first search"])  # int, NA

In [ ]:
from vaep.pandas import get_unique_non_unique_columns
columns = get_unique_non_unique_columns(mq_all_summaries.df)
mq_all_summaries.df[columns.unique]

In [ ]:
mq_all_summaries.df[columns.unique].dtypes

In [ ]:
mq_all_summaries.df[columns.unique].iloc[0, :]

## Analysis of completeness

In [ ]:
class col_summary:
    MS1 = 'MS'
    MS2 = 'MS/MS' 
    MS2_identified  = 'MS/MS Identified'
    peptides_identified = 'Peptide Sequences Identified'

if mq_all_summaries.df is None:
    raise ValueError("No data assigned")
    
MS_spectra = mq_all_summaries.df[[col_summary.MS1, col_summary.MS2, col_summary.MS2_identified, col_summary.peptides_identified]]

def compute_summary(threshold_identified):
    mask  = MS_spectra[col_summary.peptides_identified] >= threshold_identified
    display(MS_spectra.loc[mask].describe(np.linspace(0.05, 0.95, 10)))

w_ions_range = widgets.IntSlider(value=15_000, min=.0, max=MS_spectra[col_summary.peptides_identified].max())
display(widgets.interactive(compute_summary, threshold_identified=w_ions_range))

List of samples without any identified peptides:

In [ ]:
mask = (MS_spectra < 1).any(axis=1)
MS_spectra.loc[mask]

## Export selected list of quantified samples

Based on threshold, save a list of the specified samples

In [ ]:
dump_dict = {'threshold': int(w_ions_range.value)}
mask = MS_spectra[col_summary.peptides_identified] >= w_ions_range.value
dump_dict['files'] = MS_spectra.loc[mask].index.to_list()

with open('data/samples_selected.yaml', 'w') as f:
    yaml.dump(dump_dict, stream=f)